# Understanding Event Data

## Introduction

Neutron-scattering data may be recorded in "event mode":
For each detected neutron a (pulse) timestamp and a time-of-flight is stored.
This notebook will develop an understanding of how do work with this type of data.

Our objective is *not* to demonstrate or develop a full reduction workflow.
Instead we *develop understanding of data structures and opportunities* that event data provides.

This tutorial contains exercises, but solutions are included directly.
We encourage you to download this notebook and run through it step by step before looking at the solutions.
Event data is a particularly challenging concept so make sure to understand every aspect before moving on.
We recommend to use a recent version of *JupyterLab*:
The solutions are included as hidden cells and shown only on demand.

We use data containing event data from the POWGEN powder diffractometer at SNS.
Note that the data has been modified for the purpose of this tutorial and is not entirely in its original state.
We begin by loading the file and plot the raw data:

In [ ]:
import scipp as sc
import scippneutron as scn
import plopp as pp

dg = scn.data.tutorial_event_data()
dg

In [ ]:
events = dg['events']
events.hist(spectrum=500, tof=400).plot()

We can see some diffraction lines, but they are oddly blurry.
There is also an artifact from the prompt-pulse visible at $4000~\mu s$.
This tutorial illustrates how event data gives us the power to understand and deal with the underlying issues.
Before we start the investigation we cover some basics of working with event data.

## Inspecting event data

As usual, to begin exploring a loaded file, we can inspect the HTML representation of a scipp object shown by Jupyter when typing a variable at the end of a cell (this can also be done using `sc.to_html(da)`, anywhere in a cell):

In [ ]:
events

We can tell that this is binned (event) data from the `dtype` of the data (usually `DataArrayView`) as well as the inline preview, denoting that this is binned data with lists of given lengths.
The meaning of these can best be understood using a graphical depiction of `da`, created using `sc.show`:

In [ ]:
sc.show(events)

Each value (yellow cube with dots) is a small table containing event parameters such as pulse time, time-of-flight, and weights (usually 1 for raw data).

**Definitions**:

1. In scipp we refer to each of these cubes (containing a table of events) as a *bin*.
   We can think of this as a bin (or bucket) containing a number of records.
2. An array of bins (such as the array a yellow cubes with dots, shown above) is referred to as *binned variable*.
   For example, `da.data` is a binned variable.
3. A data array with data given by a binned variable is referred to as *binned data*.
   Binned data is a precursor to dense or histogrammed data.

As we will see below binned data lets us do things that cannot or cannot properly be done with dense data, such as filtering or resampling.

Each bin "contains" a small table, essentially a 1-D data array.
For efficiency and consistency scipp does not actually store an individual data array for every bin.
Instead each bin is a view to a section (slice) of a long table containing all the events from all bins combined.
This explains the `dtype=DataArrayView` seen in the HTML representation above.
For many practical purposes such a view of a data arrays behaves just like any other data array.

The values of the bins can be accessed using the `values` property.
For dense data this might give us a `float` value, for binned data we obtain a table.
Here we access the 500th event list (counting from zero):

In [ ]:
events.values[500]

### Exercise

Use `sc.to_html()`, `sc.show()`, and `sc.table()` to explore and understand `da` as well as individual values of `da` such as `da.values[500]`.

## From binned data to dense data

While we often want to perform many operations on our data in event mode, a basic but important step is transformation of event data into dense data, since typically only the latter is suitable for data analysis software or plotting purposes.
There are two options we can use for this transformation, described in the following.

### Option 1: Summing bins

If the existing binning is sufficient for our purpose we may simply sum over the rows of the tables making up the bin values:

In [ ]:
da_bin_sum = events.bins.sum()

Here we used the special `bins` property of our data array to apply an operation to each of the bins in `da`.
Once we have summed the bin values there are no more bins, and the `bins` property is `None`:

In [ ]:
print(da_bin_sum.bins)

We can visualize the result, which dense (histogram) data.
Make sure to compare the representations with those obtained above for binned data (`da`):

In [ ]:
sc.to_html(da_bin_sum)
sc.show(da_bin_sum)

We can use `da_bins_sum` to, e.g., plot the total counts per spectrum by summing over the `tof` dimension:

In [ ]:
da_bin_sum.sum('tof').plot(marker='.')

Note:
In this case there is just a single time-of-flight bin so we could have used `da_bin_sum['tof', 0]` instead of `da_bin_sum.sum('tof')`.

### Option 2: Histogramming

For performance and memory reasons binned data often contains the minimum number of bins that is "necessary" for a given purpose.
In this case `da` only contains a single time-of-flight bin (essentially just as information what the lower and upper bounds are in which we can expect events), which is not practical for downstream applications such as data analysis or plotting.

Instead of simply summing over all events in a bin we may thus *histogram* data.
Note that scipp makes the distinction between binning data (preserving all events individually) and histogramming data (summing all events that fall inside a bin).

For simplicity we consider only a single spectrum:

In [ ]:
spec = events['spectrum', 8050]
sc.show(spec)

In [ ]:
sc.table(spec.values[0]['event', :5])

Note the chained slicing above:
We access the zeroth event list and select the first 5 slices along the `event` dimension (which is the only dimension, since the event list is a 1-D table).

We use one of the [scipp functions for creating a new variable](https://scipp.github.io/reference/creation-functions.html) to define the desired bin edge of our histogram.
In this case we use `sc.linspace` (another useful option is `sc.geomspace`):

In [ ]:
tof_edges = sc.linspace(dim='tof', start=18.0, stop=17000, num=101, unit='us')
spec.hist(tof=tof_edges).plot()

#### Exercise

Change `tof_edges` to control what is plotted:

- Change the number of bins, e.g., to a finer resolution.
- Change the start and stop of the edges to plot only a smaller time-of-flight region.

#### Solution

In [ ]:
tof_edges = sc.linspace(dim='tof', start=2000.0, stop=15000, num=201, unit='us')
spec.hist(tof=tof_edges).plot()

## Masking event data — Binning by existing parameters

While quickly converting binned (event) data into dense (histogrammed) data has its applications, we may typically want to work with binned data as long as possible.
We have learned in [Working with masks](2_working-with-masks.ipynb) how to mask dense, histogrammed, data.
How can we mask a time-of-flight region, e.g., to mask a prompt-pulse, in *event mode*?

Let us sum all spectra and define a dummy data array (named `prompt`) to illustrate the objective:

In [ ]:
spec = events['spectrum', 8050].copy()
# Start and stop are fictitious and this prompt pulse is not actually present in the raw data from SNS
prompt_start = 4000.0 * sc.Unit('us')
prompt_stop = 5000.0 * sc.Unit('us')
prompt_tof_edges = sc.sort(
    sc.concat([spec.coords['tof'], prompt_start, prompt_stop], 'tof'), 'tof'
)
prompt = sc.DataArray(
    data=sc.array(dims=['tof'], values=[0, 11000, 0], unit='counts'),
    coords={'tof': prompt_tof_edges},
)
spec_hist = events.bins.concat('spectrum').hist(tof=1701)
sc.plot({'spec': spec_hist, 'prompt': prompt})

### Masking events

We now want to mask out the prompt-pulse, i.e., the peak with exponential falloff inside the region where `prompt` in the figure above is nonzero.

We can do so by checking (for every event) whether the time-of-flight is within the region covered by the prompt-pulse.
As above, we first consider only a single spectrum.
The result can be stored as a new mask:

In [ ]:
spec1 = events['spectrum', 8050].copy()  # copy since we do some modifications below
event_tof = spec.bins.coords['tof']
mask = (prompt_start <= event_tof) & (event_tof < prompt_stop)
spec1.bins.masks['prompt_pulse'] = mask
sc.plot(
    {
        'original': events['spectrum', 8050].hist(tof=100),
        'prompt_mask': spec1.hist(tof=100),
    },
    errorbars=False,
)

Here we have used the `bins` property once more.
Take note of the following:

- We can access coords "inside" the bins using the `coords` dict provided by the `bins` property.
  This provides access to "columns" of the event tables held by the bins such as `spec.bins.coords['tof']`.
- We can do arithmetic (or other) computation with these "columns", in this case comparing with scalar (non-binned) variables.
- New "columns" can be added, in this case we add a new mask column via `spec.bins.masks`.

**Definitions**:

For a data array `da` we refer to
- coordinates such as `da.coords['tof']` as *bin coordinate* and
- coordinates such as `da.bins.coords['tof']` as *event coordinate*.

The table representation (`sc.table`) and `sc.show` illustrate this process of masking:

In [ ]:
sc.table(spec1.values[0]['event', :5])
sc.show(spec1)

We have added a new column to the event table, defining *for every event* whether it is masked or not.

The generally recommended solution is different though, since masking individual events has unnecessary overhead and forces masks to be applied when converting to dense data.
A better approach is described in the next section.

#### Exercise

To get familiar with the `bins` property, try the following:

- Compute the neutron velocities for all events in `spec1`.
  Note: The total flight path length can be computed using `scn.Ltotal(spec1, scatter=True)`.
- Add the neutron velocity as a new event coordinate.
- Use, e.g., `sc.show` to verify that the coordinate has been added as expected.
- Use `del` to remove the event coordinate and verify that the coordinate was indeed removed.

#### Solution

In [ ]:
spec1.bins.coords['v'] = scn.Ltotal(spec1, scatter=True) / spec1.bins.coords['tof']
sc.show(spec1)
sc.to_html(spec1.values[0])
del spec1.bins.coords['v']
sc.to_html(spec1.values[0])

### Masking bins

Rather than masking individual events, let us simply "sort" the events depending on whether they fall below, inside, or above the region of the prompt-pulse.
We do not actually need to fully sort the events but rather use a *binning* procedure, using `sc.bin`:

In [ ]:
spec2 = events['spectrum', 8050].copy()  # copy since we do some modifications below
spec2 = sc.bin(spec2, tof=prompt_tof_edges)
prompt_mask = sc.array(dims=spec2.dims, values=[False, True, False])
spec2.masks['prompt_pulse'] = prompt_mask
sc.show(spec2)

Compare this to the graphical representation for `spec1` above and to the figure of the prompt pulse.
The start and stop of the prompt pulse are used to cut the total time-of-flight interval into three sections (bins).
The center bin is masked:

In [ ]:
spec2.masks['prompt_pulse']

#### Bonus question

Why did we not use a fine binning, e.g., with 1000 time-of-flight bins and mask a range of bins, similar to how it would be done for histogrammed (non-event) data?

#### Solution

- This would add a lot of over overhead from handling many bins.
  If our instrument had 1.000.000 pixels we would have 1.000.000.000 bins, which comes with significant memory overhead but first and foremost compute overhead.

## Binning by new parameters

After having understood how to mask a prompt-pulse we continue by considering the proton-charge log:

In [ ]:
proton_charge = dg['proton_charge']
proton_charge.plot(marker='.')

To mask a time-of-flight range, we have used `sc.bin` to adapt the binning along the *existing* `tof` dimension.
`sc.bin` can also be used to introduce binning along *new* dimension.
We define our desired pulse-time edges:

In [ ]:
tmin = proton_charge.coords['time'].min()
tmax = proton_charge.coords['time'].max()
pulse_time = sc.arange(
    dim='pulse_time',
    start=tmin.value,
    stop=tmax.value,
    step=(tmax.value - tmin.value) / 10,
)
pulse_time

As above we work with a single spectrum for now and then use `sc.bin`.
The result has two dimensions, `tof` and `pulse_time`:

In [ ]:
spec = events['spectrum', 8050]
binned_spec = spec.bin(pulse_time=pulse_time)
binned_spec

We can plot the binned spectrum, resulting in a 2-D plot:

In [ ]:
binned_spec.hist(tof=20, pulse_time=100).plot()

We may also ignore the `tof` dimension if we are simply interested in the time-evolution of the counts in this spectrum.
We can do so by concatenating all bins along the `tof` dimension as follows:

In [ ]:
binned_spec.bins.concat('tof').hist(pulse_time=100).plot(errorbars=False)

### Exercise

Using the same approach as for masking a time-of-flight bin in the previous section, mask the time period starting at about 16:30 where the proton charge is very low.

- Define appropriate edges for pulse time (use as few bins as possible, not the 10 pulse-time bins from the binning example above).
- Use `sc.bin` to apply the new binning.
  Make sure to combine this with the time-of-flight binning to mask the prompt pulse.
- Set an appropriate bin mask.
- Plot the result to confirm that the mask is set and defined as expected.

Note:
In practice masking bad pulses would usually be done on a pulse-by-pulse basis.
This requires a slightly more complex approach and is beyond the scope of this introduction.

Hint:
Pulse time is stored as `datetime64`.
A simple way to create these is using an offset from a know start time such as `tmin`:

In [ ]:
tmin + sc.to_unit(sc.scalar(7, unit='min'), 'ns')

### Solution

In [ ]:
pulse_time_edges = tmin + sc.to_unit(
    sc.array(dims=['pulse_time'], values=[0, 43, 55, 92], unit='min'), 'ns'
)
# Alternative solution to creating edges:
# t1 = tmin + sc.to_unit(43 * sc.Unit('min'), 'ns')
# t2 = tmin + sc.to_unit(55 * sc.Unit('min'), 'ns')
# pulse_time_edges = sc.array(dims=['pulse_time'], unit='ns', values=[tmin.value, t1.value, t2.value, tmax.value])

pulse_time_mask = sc.array(dims=['pulse_time'], values=[False, True, False])
binned_spec = spec.bin(tof=prompt_tof_edges, pulse_time=pulse_time_edges)
binned_spec.masks['prompt_pulse'] = prompt_mask
binned_spec.masks['bad_beam'] = pulse_time_mask
binned_spec.hist(tof=20, pulse_time=100).plot()

In [ ]:
sc.show(binned_spec)

## Higher dimensions and cuts

For purposes of plotting, fitting, or data analysis in general we will typically need to convert binned data to dense data.
We discussed the basic options for this in [From binned data to dense data](#From-binned-data-to-dense-data).
In particular when dealing with higher-dimensional data these options may not be sufficient.
For example we may want to:

- Create a 1-D or 2-D cut through a 3-D volume.
- Create a 2-D cut but integrate over an interval in the remaining dimension.
- Create multi-dimensional cuts that are not aligned with existing binning.

All of the above can be achieved using tools we have already used, but not all of them are covered in this tutorial.

### Exercise

Adapt the above code used for binning and masking the *single spectrum* (`spec`) along `pulse_time` and `tof` to the *full data array* (`da`).

Hint: This is trivial.

### Solution

In [ ]:
%matplotlib widget
binned_da = events.bin(tof=prompt_tof_edges, pulse_time=pulse_time_edges)
binned_da.masks['prompt_pulse'] = prompt_mask
binned_da.masks['bad_beam'] = pulse_time_mask
pp.slicer(
    binned_da.transpose(['pulse_time', 'spectrum', 'tof']).hist(spectrum=500, tof=400)
)

### Removing binned dimensions

Let us now convert our data to $d$-spacing (interplanar lattice spacing).
This works just like for dense data:

In [ ]:
import scippneutron as scn

dspacing_graph = {
    **scn.conversion.graph.beamline.beamline(scatter=True),
    **scn.conversion.graph.tof.elastic_dspacing('tof'),
}
da_dspacing = binned_da.transform_coords('dspacing', graph=dspacing_graph)
# `dspacing` is now a multi-dimensional coordinate, which makes plotting inconvenient, so we adapt the binning
dspacing = sc.linspace(dim='dspacing', unit='Angstrom', start=0.0, stop=3.0, num=4)
da_dspacing = sc.bin(da_dspacing, dspacing=dspacing)
da_dspacing

In [ ]:
pp.slicer(
    da_dspacing.transpose(['pulse_time', 'spectrum', 'dspacing']).hist(
        spectrum=500, dspacing=400
    )
)

After conversion to $d$-spacing we may want to combine data from all spectra.
For dense data we would have used `da_dspacing.sum('spectrum')`.
For binned data this is not possible (since the events list in every spectrum have different lengths).
Instead we need to *concatenate* the lists from bins across spectra:

In [ ]:
da_dspacing_total = da_dspacing.bins.concat('spectrum')
da_dspacing_total.hist(dspacing=400, pulse_time=500).plot()

If we zoom in we can now understand the reason for the blurry diffraction lines observed at the very start of this tutorial:
The lines are not horizontal, i.e., $d$-spacing appears to depend on the pulse time.
Note that the effect depicted here was added artificially for the purpose of this tutorial and is likely much larger than what could be observed in practice from changes in sample environment parameters such as (pressure or temperature).

Our data has three pulse-time bins (setup earlier for masking an area with low proton charge).
We can thus use slicing to compare the diffraction pattern at different times (used as a stand-in for a changing sample-environment parameter):

In [ ]:
tmp = da_dspacing_total
lines = {}
lines['total'] = tmp.bins.concat('pulse_time')
for i in 0, 2:
    lines[f'interval{i}'] = tmp['pulse_time', i]
sc.plot({k: line.hist(dspacing=1000) for k, line in lines.items()}, norm='log')

How can we extract thinner `pulse_time` slices?
We can use `sc.bin` with finer pulse-time binning, such that individual slices are thinner.
Instead of manually setting up a `dict` of slices we can use `sc.collapse`:

In [ ]:
pulse_time = sc.arange(
    dim='pulse_time',
    start=tmin.value,
    stop=tmax.value,
    step=(tmax.value - tmin.value) / 10,
)
split = da_dspacing_total.bin(pulse_time=pulse_time)
sc.plot(sc.collapse(split.hist(dspacing=1000), keep='dspacing'))

### Making a 1-D cut

Instead of summing over all spectra we may want to group spectra based on a $2\theta$ interval they fall into.
$2\theta$ was computed earlier as a side effect of the conversion from time-of-flight to $d$-spacing:

In [ ]:
da_dspacing.coords['two_theta']

We can then define the boundaries we want to use for our "cut".
Here we use just a single bin in each of the three dimensions:

In [ ]:
two_theta_cut = sc.linspace(dim='two_theta', unit='rad', start=0.4, stop=1.0, num=2)
# Do not use many bins, fewer is better for performance
dspacing_cut = sc.linspace(dim='dspacing', unit='Angstrom', start=0.0, stop=2.0, num=2)
pulse_time_cut = tmin + sc.to_unit(
    sc.array(dims=['pulse_time'], unit='s', values=[0, 10 * 60]), 'ns'
)

cut = da_dspacing.bin(
    two_theta=two_theta_cut, dspacing=dspacing_cut, pulse_time=pulse_time_cut
)
cut

We can then use slicing (to remove unwanted dimensions) and `sc.histogram` to get the desired binning:

In [ ]:
cut = cut['pulse_time', 0]  # squeeze pulse time (dim of length 1)
cut = cut['two_theta', 0]  # squeeze two_theta (dim of length 1)
cut = sc.hist(cut, dspacing=1000)
cut.plot()

#### Exercise

- Adjust the start and stop values in the cut edges above to adjust the "thickness" of the cut.
- Adjust the edges used for histogramming.

### Making a 2-D cut

#### Exercise

- Adapt the code of the 1-D cut to create 100 `two_theta` bins.
- Make a 2-D plot (with `dspacing` and `two_theta` on the axes).

#### Solution

In [ ]:
two_theta_cut = sc.linspace(dim='two_theta', unit='rad', start=0.4, stop=1.0, num=101)
dspacing_cut = sc.linspace(dim='dspacing', unit='Angstrom', start=0.0, stop=2.0, num=2)
pulse_time_cut = tmin + sc.array(dims=['pulse_time'], unit='s', values=[0, 10 * 60]).to(
    unit='ns'
)
cut = da_dspacing.bin(
    two_theta=two_theta_cut, dspacing=dspacing_cut, pulse_time=pulse_time_cut
)
cut = cut['pulse_time', 0]  # squeeze pulse time (dim of length 1)
dspacing_edges = sc.linspace(
    dim='dspacing', unit='Angstrom', start=0.0, stop=2.0, num=1000
)
cut = cut.hist(dspacing=dspacing_edges)
cut.plot()